# Creating Melbourne Business Database

## Extract Data From API

In [2]:
# Dependencies
import pandas as pd, requests, json
from sodapy import Socrata
from pandas_geojson import to_geojson
from sqlalchemy import create_engine

In [3]:
# API call
client = Socrata("data.melbourne.vic.gov.au", None)
results = client.get("xt2y-tnn9", limit = 100000)

In [4]:
# Create dataframe
results_df = pd.DataFrame.from_records(results)
results_df.to_csv('data/melbourne_business_data.csv')
results_df.head(2)

,census_year,block_id,property_id,base_property_id,street_address,clue_small_area,trading_name,industry_anzsic4_code,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate,geocoded_column
0,2002,2,111467,103973,0 King Street MELBOURNE 3000,Melbourne (CBD),469-479 King Street MELBOURNE 3000,8921,Zoological and Botanical Gardens Operation,Seats - Indoor,113,144.9595,-37.8206,"{'type': 'Point', 'coordinates': [144.9595, -3..."
1,2002,2,111467,103973,0 King Street MELBOURNE 3000,Melbourne (CBD),469-479 King Street MELBOURNE 3000,8921,Zoological and Botanical Gardens Operation,Seats - Outdoor,38,144.9595,-37.8206,"{'type': 'Point', 'coordinates': [144.9595, -3..."


In [5]:
len(results_df)

54052

## Transform Data

In [6]:
# remove column
transformed_result_df = results_df[["census_year", "clue_small_area", "trading_name", "industry_anzsic4_description", 
                                    "seating_type", "number_of_seats","x_coordinate", "y_coordinate"]]

# drop na
transformed_result_df = transformed_result_df.dropna()
print(len(transformed_result_df))
transformed_result_df.dtypes

53990


census_year                     object
clue_small_area                 object
trading_name                    object
industry_anzsic4_description    object
seating_type                    object
number_of_seats                 object
x_coordinate                    object
y_coordinate                    object
dtype: object

In [7]:
# change data type
transformed_result_df ['census_year'] = transformed_result_df ['census_year'].astype('int')
transformed_result_df ['number_of_seats'] = transformed_result_df ['number_of_seats'].astype('int')
transformed_result_df ['x_coordinate'] = transformed_result_df ['x_coordinate'].astype('float')
transformed_result_df ['y_coordinate'] = transformed_result_df ['y_coordinate'].astype('float')
transformed_result_df.dtypes
transformed_result_df.to_csv('data/cleaned_melbourne_business_data.csv')

In [26]:
# filter by year
transformed_result_df = transformed_result_df.loc [(transformed_result_df['census_year'] >= 2015),:]
transformed_result_df = transformed_result_df.loc [(transformed_result_df['seating_type'] == "Seats - Indoor"),:]

In [28]:
transformed_result_df.head()

,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
33173,2015,Melbourne (CBD),26 Rebecca Walk MELBOURNE 3000,Cafes and Restaurants,Seats - Indoor,32,144.9575,-37.8211
33176,2015,Melbourne (CBD),20 Rebecca Walk MELBOURNE 3000,Takeaway Food Services,Seats - Indoor,4,144.9575,-37.8211
33177,2015,Melbourne (CBD),"Part Ground , 525 Flinders Street MELBOURNE 3000",Cafes and Restaurants,Seats - Indoor,50,144.9572,-37.8209
33178,2015,Melbourne (CBD),557 Flinders Street MELBOURNE 3000,Takeaway Food Services,Seats - Indoor,6,144.9565,-37.8210
33179,2015,Melbourne (CBD),547 Flinders Street MELBOURNE 3000,Cafes and Restaurants,Seats - Indoor,44,144.9565,-37.8210


In [8]:
# create geojson
geo_json = to_geojson(df=transformed_result_df, lat='y_coordinate', lon='x_coordinate',properties=["census_year", "clue_small_area", "trading_name", "industry_anzsic4_description", "seating_type", "number_of_seats"])

In [9]:
# save geojson file
from geojson import dump
with open('data/melbourne_business_da.geojson', 'w') as output_file:
    dump(geo_json,output_file)

## Load dataframe to database 

In [10]:
# create connection
connection = "postgres:David$1986@localhost:5432/Melbourne_Business_db"
engine = create_engine(f'postgresql://{connection}')

In [11]:
# Confirm tables
engine.table_names()

c:\Users\david\Anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['melbourne_business']

In [12]:
# load dataframe
transformed_result_df.to_sql(name="melbourne_business", con=engine, if_exists='append', index=False)